In [1]:
import sys
import os

#library_path is the path where the Optimpy library is located.
library_path = "/home/dell/Documentos/Git_proejcts/pyristic/"
#library_path = "/Users/adrianamenchacamendez/Documentos/enes_morelia/papime/optimizacion-con-metaheuristicas/"
sys.path.append(os.path.abspath(library_path))

In [2]:
from pyristic.heuristic.Tabu_search import TabuSearch as TS
from pyristic.heuristic.SimulatedAnnealing_search import SimulatedAnnealing as SA
from pyristic.heuristic.GeneticAlgorithm_search import Genetic as GA
from pyristic.utils.helpers import *
from pyristic.utils.operators import selection,mutation,crossover

In [3]:
from pprint import pprint
import numpy as np 
import random 
import copy 

In [4]:
np.random.seed(10)

## Problema de la mochila

In [5]:
class knapsackFunction:
    def __init__(self, benefits: (list, np.ndarray)):
        self.benefits = np.array(benefits)
        
    def __call__(self,X: np.ndarray):
        return -1 * np.dot(X,self.benefits)

class knapsackAptitude:
    def __init__(self, benefits: (list, np.ndarray)):
        self.benefits = np.array(benefits)
        
    def __call__(self,X:np.ndarray):
        return np.dot(X,self.benefits) 

           
class knapsackConstraint:
    
    def __init__(self, weights: (list,np.ndarray), cappacity):
        self.weights = np.array(weights)
        self.cappacity = cappacity
        
    def __call__(self,X: np.ndarray):
        result  = np.dot(X, self.weights)
        self.__doc__="{} <= {}".format(result,self.cappacity)
        return result <= self.cappacity
    


#### Instancia 1

In [6]:
n1 = 50
p1 = [60, 52, 90, 57, 45, 64, 60, 45, 63, 94, 44, 90, 66, 64, 32, 39, 91, 40, 73, 61, 82, 94, 39, 68, 94, 98, 80, 79, 73, 99, 49, 56, 69, 49, 82, 99, 65, 34, 31, 85, 67, 62, 56, 38, 54, 81, 98, 63, 48, 83]
w1 = [38, 20, 21, 21, 37, 28, 32, 30, 33, 35, 29, 32, 35, 24, 28, 29, 22, 34, 31, 36, 36, 28, 38, 25, 38, 37, 20, 23, 39, 31, 27, 20, 38, 38, 36, 28, 39, 22, 23, 22, 21, 24, 23, 33, 31, 30, 32, 30, 22, 37]
c1 = 500

In [7]:
f1 = knapsackFunction(p1)
g1 = knapsackConstraint(w1,c1)

#### Instancia 2

In [8]:
n2 = 100
p2 =  np.random.randint(10,100,n2)
w2 =  np.random.randint(15,80,n2)
c2 =  1000

In [9]:
f2 = knapsackFunction(p2)
g2 = knapsackConstraint(w2,c2)


#### Instancia 3

In [10]:
n3 = 150
p3 =  np.random.randint(10,100,n3)
w3 =  np.random.randint(15,80,n3)
c3 =  1500

In [11]:
f3 = knapsackFunction(p3)
g3 = knapsackConstraint(w3,c3)

### Búsqueda tabú configuración

#### Solución inicial

In [12]:
def getInitialSolution(n,p,w,c):
    #Empty backpack
    x = [0 for i in range(n)]
    weight_x = 0
    
    #Random order to insert objects.
    objects = list(range(n))
    np.random.shuffle(objects)
    
    for o in  objects:
        #Check the constraint about capacity.
        if weight_x + w[o] <= c:
            x[o] = 1
            weight_x += w[o]
            
    return np.array(x)

#### Configuración metaheurística

In [13]:
class knapsack_TS(TS):
    
    def __init__(self, f_ : function_type , constraints_: list):
        super().__init__(f_,constraints_)
        
        
    def get_neighbors(self, x : np.ndarray,**kwargs) -> list:   
        neighbors_list = []

        for i in range(len(x)):
            x[i] ^= 1 #1
            neighbors_list+=[copy.deepcopy(x)]
            x[i] ^= 1 
            
        return neighbors_list
        
    def encode_change(self, neighbor : (list,np.ndarray), x : (list,np.ndarray),**kwargs) -> list: #2
        
        x_ = [None,None]
        
        for i in range(len(x)):
            if x[i] != neighbor[i]:
                return [i,neighbor[i]]
            
        return x_

### Búsqueda recocido simulado

#### Solución inicial

In [14]:
class initializer:

    def __init__(self, n,p,w,c):
        self.n = n
        self.p = p
        self.w = w
        self.c = c
        
        self.initGISP()
        self.getSolutionByGISP()
    
    def initGISP(self):
        self.GISP_Arr=[]
        for i in range(self.n):
            self.GISP_Arr.append((self.p[i]/self.w[i],i))

        self.GISP_Arr.sort(reverse=True)
    
    def getSolutionByGISP(self):
        X = [0 for i in range(self.n)]
        current_weight = 0
        for i in range(self.n):
            ind = self.GISP_Arr[i][1]
            if current_weight+ self.w[ind] <= self.c:
                current_weight+=self.w[ind]
                X[ind] = 1
        self.solution =  np.array(X)


#### Configuración metaheurística

In [15]:
class knapsack_SA(SA):

    def __init__(self, f_ : function_type , constraints_: list, n,p,w,c, GISP_Arr):
        super().__init__(f_,constraints_)
        self.n = n
        self.p = p
        self.w = w
        self.c = c
        self.GISP_Arr = GISP_Arr
        
    def generate_neighbor(self, x: np.ndarray) -> np.ndarray:
        x_ = x.copy()
        N = len(x_)
        ind = random.randint(0, N-1)
        x_[ind] ^= 1
        return x_
    
    def repair_neighbor(self, x: np.ndarray) -> np.ndarray:
        #Get the total weight
        total_weight=0
        for i in range(self.n):
            if x[i]:
                total_weight += self.w[i]
        
        for i in range(self.n):
            ind = self.GISP_Arr[self.n - (1+i)][1] #Lowest
            
            if x[ind]:
                total_weight -= self.w[ind]
                x[ind] = 0
            
            if total_weight <= self.c:
                break
                
    def improve_neighbor(self, x: np.ndarray) -> np.ndarray:
        
        #Get the total weight
        total_weight=0
        for i in range(self.n):
            if x[i]:
                total_weight += self.w[i]
                
        for i in range(self.n):
            ind = self.GISP_Arr[i][1]
            if total_weight + self.w[ind] > self.c:
                continue
                
            if x[ind] == 0 :
                total_weight += self.w[ind]
                x[ind] = 1
                
    def get_neighbor(self, x : np.ndarray) -> np.ndarray: 
        
        neighbor_ = self.generate_neighbor(x)
            
        if(self.is_valid(neighbor_)):
            return neighbor_
        
        #RI strategy
        self.repair_neighbor(neighbor_)
        self.improve_neighbor(neighbor_)
        
        return neighbor_

### Búsqueda Algoritmos genéticos

#### Configuración de operadores

In [16]:
configuration_knapsack = (GeneticConfig()
                             .cross(crossover.n_point_crossover(2))
                             .mutate(mutation.binary_mutator(.1))
                             .survivor_selection(selection.merge_selector())
                             .parent_selection(selection.tournament_sampler(chunks_=5,prob_=0.8))
                             )

#### Configuración de metaheurística

In [17]:
def fixer(individual,weights, cappacity):
    indices = []
    current_weight = 0
    for index,item in enumerate(individual):
        if item == 1:
            indices.append(index)
            current_weight += weights[index]

    permutation_indices = np.random.permutation(indices)
    new_solution = np.zeros(len(individual))
    new_solution[indices] = 1

    for i in permutation_indices:
        if current_weight <= cappacity:
            break
        current_weight -= weights[i]
        new_solution[i] = 0

    return new_solution     
    
class knapsack_GA(GA):
    def __init__(self,  function,\
                        decision_variables,\
                        constraints,\
                        config,\
                        n,\
                        p,\
                        w,\
                        c):
        super().__init__(function, decision_variables, constraints, [], config)
        self.n = n
        self.p = p
        self.w = w
        self.c = c
        
    def initialize_population(self, **kwargs) -> np.ndarray:
        individuals = []
        
        for i in range(self.logger['population_size']):
            individual_ = np.random.randint(2, size= self.n)
            if self.is_invalid(individual_):
                individuals += [fixer(individual_, self.w, self.c)]
            else:
                individuals += [individual_]
        return np.array(individuals)
 
    def fixer(self, ind: int) -> np.ndarray:
        return fixer(self.logger['offspring_population_x'][ind],self.w, self.c)

### Resultados 

#### Instancia 1

In [18]:
AG_instancia1  = knapsack_GA(     
                    function= knapsackAptitude(p1),\
                    decision_variables=n1,\
                    constraints= [g1],\
                    config=configuration_knapsack,\
                    n = n1,\
                    p = p1,\
                    w = w1,\
                    c = c1
                )

In [19]:
args = (300,150,0.9, False)
statistics = get_stats(AG_instancia1, 30, args,{},showAll=False)



/home/dell/Documentos/Git_proejcts/pyristic/pyristic/utils/operators/crossover.py:110: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  new_population[2*row][start:final]    = X[parent_ind2[row]][start:final]


In [20]:
pprint(statistics)

{'Best solution': {'f': 1521.0,
                   'x': array([0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.])},
 'Mean': 1526.8,
 'Median': 1527.0,
 'Standard deviation': 4.361956747454823,
 'Worst solution': {'f': 1540.0,
                    'x': array([0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.])}}


In [21]:
initial_solution = getInitialSolution(n1,p1,w1,c1)
TS_instancia1 = knapsack_TS(
    f1,
    [g1]
    )

In [22]:
args = (initial_solution,500,n1//2,False)
statistics = get_stats(TS_instancia1, 30, args,showAll=False)

In [23]:
pprint(statistics)

{'Best solution': {'f': -999,
                   'x': array([1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1])},
 'Mean': -999.0,
 'Median': -999.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': -999,
                    'x': array([1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1])}}


In [24]:
initial_solution = initializer(n1,p1,w1,c1)
SA_instancia1 = knapsack_SA(f1, [g1],n1,p1,w1,c1,initial_solution.GISP_Arr)

In [25]:
args = (initial_solution.solution,1000.0,0.01)
statistics = get_stats(SA_instancia1, 30, args,showAll=False)

In [26]:
pprint(statistics)

{'Best solution': {'f': -1555,
                   'x': array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0])},
 'Mean': -1549.0666666666666,
 'Median': -1550.5,
 'Standard deviation': 4.545571715661542,
 'Worst solution': {'f': -1542,
                    'x': array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0])}}


#### Instancia 2

In [27]:
AG_instancia2  = knapsack_GA(     
                    function= knapsackAptitude(p2),\
                    decision_variables=n2,\
                    constraints= [g2],\
                    config=configuration_knapsack,\
                    n = n2,\
                    p = p2,\
                    w = w2,\
                    c = c2
                )

In [28]:
args = (300,150,0.9, False)
statistics = get_stats(AG_instancia2, 30, args,{},showAll=False)

In [29]:
pprint(statistics)

{'Best solution': {'f': 2134.0,
                   'x': array([1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.])},
 'Mean': 2148.366666666667,
 'Median': 2144.0,
 'Standard deviation': 12.177803122439158,
 'Worst solution': {'f': 2173.0,
                    'x': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 

In [30]:
initial_solution = getInitialSolution(n2,p2,w2,c2)
TS_instancia2 = knapsack_TS(
    f2,
    [g2]
    )

In [31]:
args = (initial_solution,500,n2//2,False)
statistics = get_stats(TS_instancia2, 30, args,showAll=False)

In [32]:
pprint(statistics)

{'Best solution': {'f': -1098,
                   'x': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])},
 'Mean': -1098.0,
 'Median': -1098.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': -1098,
                    'x': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])}}


In [33]:
initial_solution = initializer(n2,p2,w2,c2)
SA_instancia1 = knapsack_SA(f2, [g2],n2,p2,w2,c2,initial_solution.GISP_Arr)

In [34]:
args = (initial_solution.solution,1000.0,0.01)
statistics = get_stats(SA_instancia1, 30, args,showAll=False)

In [35]:
pprint(statistics)

{'Best solution': {'f': -2417,
                   'x': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])},
 'Mean': -2410.4666666666667,
 'Median': -2410.0,
 'Standard deviation': 5.064473209415653,
 'Worst solution': {'f': -2401,
                    'x': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])}}


#### Instancia 3

In [36]:
AG_instancia3  = knapsack_GA(     
                    function= knapsackAptitude(p3),\
                    decision_variables=n3,\
                    constraints= [g3],\
                    config=configuration_knapsack,\
                    n = n3,\
                    p = p3,\
                    w = w3,\
                    c = c3
                )

In [37]:
args = (300,150,0.9, False)
statistics = get_stats(AG_instancia3, 30, args,{},showAll=False)

In [38]:
pprint(statistics)

{'Best solution': {'f': 3101.0,
                   'x': array([1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.])},
 'Mean': 3131.8,
 'Median': 3133.0,
 'Standard deviation': 15.990413794937684,
 'Worst solution': {'f': 3168.0,
                    'x': array([1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,

In [39]:
initial_solution = getInitialSolution(n3,p3,w3,c3)
TS_instancia3 = knapsack_TS(
    f3,
    [g3]
    )

In [40]:
args = (initial_solution,500,n3//2,False)
statistics = get_stats(TS_instancia3, 30, args,showAll=False)

In [41]:
pprint(statistics)

{'Best solution': {'f': -2167,
                   'x': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])},
 'Mean': -2167.0,
 'Median': -2167.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': -2167,
                    'x': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 

In [42]:
initial_solution = initializer(n3,p3,w3,c3)
SA_instancia3 = knapsack_SA(f3, [g3],n3,p3,w3,c3,initial_solution.GISP_Arr)

In [43]:
args = (initial_solution.solution,1000.0,0.01)
statistics = get_stats(SA_instancia3, 30, args,showAll=False)

In [44]:
pprint(statistics)

{'Best solution': {'f': -3777,
                   'x': array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0])},
 'Mean': -3769.9333333333334,
 'Median': -3768.0,
 'Standard deviation': 2.8158282775923835,
 'Worst solution': {'f': -3768,
                    'x': array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
 

## Problema del agente viajero

In [45]:
class TSPFunction:
    def __init__(self, distanceMatrix: (list, np.ndarray)):
        self.distance = np.array(distanceMatrix)
        
    def __call__(self,x: np.ndarray):
        total_dist = self.distance[int(x[-1])][0] + self.distance[0][int(x[0])]
        for i in range(1,len(x)):
            u,v = int(x[i]), int(x[i-1])
            total_dist+= self.distance[u][v]

        return float(total_dist)

class TSPAptitude:
    def __init__(self,  distanceMatrix: (list, np.ndarray)):
        self.f = TSPFunction(distanceMatrix)
        
    def __call__(self,X:np.ndarray):
        return -1 * self.f(X) 

  

In [46]:
def create_travel_matrix(n, low_bound, high_bound):
    matrix = np.zeros((n,n))
    for i in range(1,n):
        for j in range(i):
            value =  np.random.randint(low_bound, high_bound)
            matrix[i][j] = value
            matrix[j][i] = value 
    return matrix

### Instancia 1

In [47]:
matrix1 = create_travel_matrix(10,10,100)
n1 = 10
f1 = TSPFunction(matrix1)

### Instancia 2


In [48]:
matrix2 = create_travel_matrix(15,10,100)
n2 = 15
f2 = TSPFunction(matrix2)

### Instancia 3

In [49]:
matrix3 = create_travel_matrix(20,10,100)
n3 = 20
f3 = TSPFunction(matrix3)

## Búsqueda Tabú

#### Solución inicial

In [50]:
def getInitialSolution(distance_matrix, total_cities):
    Solution = [0]
    remaining_cities  = list(range(1,total_cities))
    
    while len(remaining_cities) != 0:
        from_ =Solution[-1] 
        to_ = remaining_cities[0]
        dist = distance_matrix[from_][to_]
        
        for i in range(1, len(remaining_cities)):
            distance = distance_matrix[from_][remaining_cities[i]]
            if distance < dist:
                to_ = remaining_cities[i]
                dist = distance
        Solution.append(to_)
        ind = remaining_cities.index(to_)
        remaining_cities.pop(ind)
    return Solution

#### Configuración de metaheuristica

In [51]:
class Tabu_Salesman_list:
    def __init__(self,timer):
        self.__TB = {}
        self.timer = timer
    
    def reset(self,timer) -> None:
        self.__TB = {}
        self.timer = timer
        
    def update(self) -> None:
        to_pop = []
        for key in self.__TB:
            if self.__TB[key]-1 == 0:
                to_pop.append(key)
            else:
                self.__TB[key]-=1
        for key in to_pop:
            self.__TB.pop(key)
        
    #x has [p,v,step], we are only interested in v (value)
    def push(self, x : list ) -> None:
        self.__TB[x[1]] = self.timer
        

    def find(self, x : list) -> bool:
        return x[1] in self.__TB
        

In [52]:
class TSP_TS(TS):

    def __init__(self, f_ : function_type , TabuStorage):
        super().__init__(f_,[],TabuStorage)
        

    def get_neighbors(self, x : np.ndarray,**kwargs) -> list: 
        
        neighbors_list = []
        
        ind = random.randint(1,len(x)-1)
        while  self.TL.find([-1,x[ind]]):
            ind = random.randint(1,len(x)-1)
        v = x[ind]
        x_tmp = list(x[v != x])
        for i in range(1, len(x)):
            if ind == i:
                continue
            neighbors_list += [ x_tmp[:i] + [v] + x_tmp[i:]]
            
        return neighbors_list


    def encode_change(self, neighbor : (list,np.ndarray), x : (list,np.ndarray),**kwargs) -> list: #2
        
        x_p ={x[i] : i for i in range(len(x))}
        n_p = {neighbor[i]: i for i in range(len(x))}
        ind = -1
        max_dist = -1
        value = -1
        for i in range(1, len(x)):
            v = x[i]
            dist = abs(x_p[v] - n_p[v])
            if dist > max_dist:
                ind = i
                max_dist = dist
                value = v
       
        return [ind , value]

## Búsqueda Recocido simulado

#### Configuración metaheurística

In [53]:
class TSP_SA(SA):

    def __init__(self, f_ : function_type ):
        super().__init__(f_,[])

    def get_neighbor(self, x : np.ndarray) -> np.ndarray: 
        
        x_ = x.copy()
        N = len(x_)
        index1 = random.randint(1, N-1)
        index2 = random.randint(1, N-1)
    
        while index2 == index1:
            index2 = random.randint(1, N-1)
    
        v = x[index1]
        x_ = list(x_[v != x_])
        x_ = x_[:index2] + [v] + x_[index2:]
        return np.array(x_)


## Búsqueda Algoritmos genéticos

#### Configuración de operadores

In [54]:
configuration_TSP = (GeneticConfig()
                         .cross(crossover.permutation_order_crossover())
                         .mutate(mutation.insertion_mutator(1))
                         .survivor_selection(selection.merge_selector())
                         .parent_selection(selection.tournament_sampler(chunks_=5,prob_=0.8))
                         .fixer_invalide_solutions(NoneFixer()))


#### Configuración de metaheurística

In [55]:
class TSP_GA(GA):
    def __init__(self,  function,\
                        decision_variables:int,\
                        config = None):
        super().__init__(function, decision_variables, [],[], config)
    
    def initialize_population(self, **kwargs) -> np.ndarray:
        individuals = []
        
        for i in range(self.logger['population_size']):
            individuals += [np.random.permutation(range(1,self.Decision_variables))]
            
        return np.array(individuals)

### Resultados

#### Instancia 1

In [56]:
TS_instancia1 = TSP_TS(f1,Tabu_Salesman_list(n1//2))
init_path = np.array(getInitialSolution(matrix1,n1))

In [57]:
args = (init_path, 100, n1//2, False)
statistics = get_stats(TS_instancia1, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 249.0, 'x': array([0, 9, 1, 5, 4, 8, 6, 7, 2, 3])},
 'Mean': 249.0,
 'Median': 249.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': 249.0, 'x': array([0, 9, 1, 5, 4, 8, 6, 7, 2, 3])}}


In [58]:
SA_instancia1 = TSP_SA(f1)

In [59]:
args = (init_path, 1000.0, 0.01)
statistics = get_stats(SA_instancia1, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 228.0, 'x': array([0, 7, 2, 3, 6, 8, 4, 5, 1, 9])},
 'Mean': 243.7,
 'Median': 249.0,
 'Standard deviation': 8.149233092751736,
 'Worst solution': {'f': 249.0, 'x': array([0, 9, 1, 5, 4, 8, 6, 7, 2, 3])}}


In [60]:
GA_instancia1 = TSP_GA(TSPAptitude(matrix1),n1,configuration_TSP)

In [61]:
args = (200, 100,0.9,False)
statistics = get_stats(GA_instancia1, 30, args, transformer=f1, showAll=False)
pprint(statistics)

/home/dell/Documentos/Git_proejcts/pyristic/pyristic/utils/operators/crossover.py:175: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "permutation_order_cross" failed type inference due to: Invalid use of type(CPUDispatcher(<function create_child at 0x7fd353b46b80>)) with parameters (array(int64, 1d, C), array(int64, 1d, C))

During: resolving callee type: type(CPUDispatcher(<function create_child at 0x7fd353b46b80>))
During: typing of call at /home/dell/Documentos/Git_proejcts/pyristic/pyristic/utils/operators/crossover.py (199)


File "pyristic/pyristic/utils/operators/crossover.py", line 199:
def permutation_order_cross(X : np.ndarray , parent_ind1:np.ndarray,\
    <source elided>
    for i in prange(num_individuals):
        new_population[2*i]  = create_child(X[parent_ind1[i]], X[parent_ind2[i]])
        ^

  @jit(nopython=False, parallel=True)
/home/dell/Documentos/Git_proejcts/pyristic/pyristic/utils/operators/crossover.py:175

/home/dell/anaconda3/envs/pyristicEnv/lib/python3.8/site-packages/numba/core/object_mode_passes.py:177: NumbaWarning: Function "permutation_order_cross" was compiled in object mode without forceobj=True.

File "pyristic/pyristic/utils/operators/crossover.py", line 198:
def permutation_order_cross(X : np.ndarray , parent_ind1:np.ndarray,\
    <source elided>

    for i in prange(num_individuals):
    ^

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/dell/anaconda3/envs/pyristicEnv/lib/python3.8/site-packages/numba/core/object_mode_passes.py:187: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "pyristic/pyristic/utils/operators/crossover.py", line 198:
def permutation_order_cross(X : np.ndarray , parent_ind1:np.nda

{'Best solution': {'f': 228.0,
                   'x': array([9., 1., 5., 4., 8., 6., 3., 2., 7.])},
 'Mean': 228.03333333333333,
 'Median': 228.0,
 'Standard deviation': 0.17950549357115012,
 'Worst solution': {'f': 229.0,
                    'x': array([7., 6., 8., 4., 5., 3., 2., 1., 9.])}}


#### Instancia 2

In [62]:
TS_instancia2 = TSP_TS(f2,Tabu_Salesman_list(n2//2))
init_path = np.array(getInitialSolution(matrix2,n2))

In [63]:
args = (init_path, 100, n2//2, False)
statistics = get_stats(TS_instancia2, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 331.0,
                   'x': array([ 0,  5, 10,  3,  1,  7,  2,  9, 14, 12,  4,  8,  6, 11, 13])},
 'Mean': 359.26666666666665,
 'Median': 367.0,
 'Standard deviation': 13.248731595473165,
 'Worst solution': {'f': 385.0,
                    'x': array([ 0,  5, 10, 12,  7,  2,  9, 13, 11,  6,  8,  4,  1,  3, 14])}}


In [64]:
SA_instancia2 = TSP_SA(f2)

In [65]:
args = (init_path, 1000.0, 0.01)
statistics = get_stats(SA_instancia2, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 297.0,
                   'x': array([ 0,  4, 12,  3,  1,  7,  2,  9, 13, 11, 14,  8,  6, 10,  5])},
 'Mean': 362.3666666666667,
 'Median': 365.0,
 'Standard deviation': 28.99595757266098,
 'Worst solution': {'f': 414.0,
                    'x': array([ 0, 14,  7,  2,  9,  1,  8,  6,  5, 10,  3, 12,  4, 11, 13])}}


In [66]:
GA_instancia2 = TSP_GA(TSPAptitude(matrix2),n2,configuration_TSP)

In [67]:
args = (200, 100,0.9,False)
statistics = get_stats(GA_instancia2, 30, args, transformer=f2, showAll=False)
pprint(statistics)

{'Best solution': {'f': 280.0,
                   'x': array([ 5., 10.,  6.,  8.,  4., 12.,  3.,  1.,  7.,  2.,  9., 14., 11.,
       13.])},
 'Mean': 318.06666666666666,
 'Median': 321.0,
 'Standard deviation': 15.541199724888967,
 'Worst solution': {'f': 344.0,
                    'x': array([ 7.,  1.,  3., 12.,  4.,  8.,  9.,  2.,  5., 10.,  6., 14., 11.,
       13.])}}


#### Instancia 3

In [68]:
TS_instancia3 = TSP_TS(f3,Tabu_Salesman_list(n3//2))
init_path = np.array(getInitialSolution(matrix3,n3))

In [69]:
args = (init_path, 100, n3//2, False)
statistics = get_stats(TS_instancia3, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 456.0,
                   'x': array([ 0, 14, 19, 18,  7,  6, 15, 10,  5,  2,  1, 11,  4,  8, 13,  9, 16,
        3, 12, 17])},
 'Mean': 462.23333333333335,
 'Median': 467.0,
 'Standard deviation': 5.4508918740167855,
 'Worst solution': {'f': 467.0,
                    'x': array([ 0, 14, 19, 18,  7,  6, 15, 10,  5,  2,  1, 11,  8,  4, 17, 13,  9,
       16,  3, 12])}}


In [70]:
SA_instancia3 = TSP_SA(f3)

In [71]:
args = (init_path, 1000.0, 0.01)
statistics = get_stats(SA_instancia3, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 447.0,
                   'x': array([ 0, 17,  4,  8,  6,  7,  9, 16,  3, 14, 13,  5,  2,  1, 11, 15, 10,
       18, 19, 12])},
 'Mean': 488.96666666666664,
 'Median': 497.0,
 'Standard deviation': 12.942136179506413,
 'Worst solution': {'f': 497.0,
                    'x': array([ 0, 14, 19, 18,  7,  6, 15, 10,  5,  2,  1, 11,  4,  8, 12, 17, 13,
        9, 16,  3])}}


In [72]:
GA_instancia3 = TSP_GA(TSPAptitude(matrix3),n3,configuration_TSP)

In [73]:
args = (200, 100,1.0,False)
statistics = get_stats(GA_instancia3, 30, args, transformer=f3, showAll=False)
pprint(statistics)

{'Best solution': {'f': 430.0,
                   'x': array([16.,  3.,  6., 15., 10., 18., 19.,  9., 13., 14.,  5.,  7.,  2.,
        1., 11.,  8.,  4., 17., 12.])},
 'Mean': 497.5,
 'Median': 500.0,
 'Standard deviation': 28.13272116237603,
 'Worst solution': {'f': 538.0,
                    'x': array([ 5., 15., 10., 17.,  4., 11.,  1.,  2.,  8., 12.,  3.,  6.,  7.,
       13., 16.,  9., 19., 18., 14.])}}
